# Kohya's Script + Lora Train `Developing`

More info: https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb

> Created by [@wibus-wee](https://github.com/wibus-wee)
>
> Reference: [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)

In [ ]:
#@title 选择模型 { display-mode: "form" }

#@markdown 选择模型
checkpoint = 'chilloutmix.safetensors' #@param ["chilloutmix.safetensors", "sunshinemix.safetensors", "grapefruitHentaiModel.safetensors"]

downloadLink = {
    'chilloutmix.safetensors': 'https://civitai.com/api/download/models/11745',
    'sunshinemix.safetensors': 'https://civitai.com/api/download/models/11752',
    'grapefruitHentaiModel.safetensors': 'https://civitai.com/api/download/models/9000'
}[checkpoint]

print("已选择模型: " + checkpoint + " <===> " + downloadLink)

%store checkpoint
%store downloadLink

In [ ]:
#@title 1. 检查 GPU & 检查环境

#@markdown 此步骤将检查你的 GPU 是否支持 xformers，同时会检查是否为 Paperspace 平台（仅针对 M4000 GPU 做了判断，因此其他的 GPU 可能在此处将会出现逻辑判断错误的情况，你可能需要自行勾选）。

import os, subprocess
paperspace_m4000 = False
#@markdown 是否为 Paperspace 平台
isPaperspace = False #@param {type:"boolean"}
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    if 'M4000' in subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8'):
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
        isPaperspace = True
    else:
        print("你正在使用的是合适的 GPU - " + subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8') + "。")
        print("平台: Paperspace" if isPaperspace else "使用平台: Colab")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = isPaperspace and '/tmp' or '/content'
%store rootDir
%store paperspace_m4000 
%store isPaperspace

In [ ]:
#@title 2. 安装训练依赖

%store -r rootDir 
%store -r checkpoint 
%store -r downloadLink

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

!git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts {rootDir}/lora-scripts
%cd {rootDir}/lora-scripts
if ariaInstalled:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {rootDir}/lora-scripts/sd-models -o {checkpoint}
else:
    !wget -c {downloadLink} -P {rootDir}/stable-diffusion-webui/models/Stable-diffusion -O {rootDir}/lora-scripts/sd-models/{checkpoint}
!chmod +x install.bash train.bash
!bash install.bash

if not paperspace_m4000:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton

In [ ]:
#@title 3. 训练选项设置

source venv/bin/activate

In [ ]:
#@title 4. 启动 GUI